In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.models import Model   
import matplotlib
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import Input, LSTM, Dense
import matplotlib.pyplot as plt
import collections
import helper

In [2]:
df = pd.read_json('train_spider.json')

In [3]:
df.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [4]:
pd.__version__

'1.3.5'

In [5]:
df.to_csv('train_spider.csv', index=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                7000 non-null   object
 1   query                7000 non-null   object
 2   query_toks           7000 non-null   object
 3   query_toks_no_value  7000 non-null   object
 4   question             7000 non-null   object
 5   question_toks        7000 non-null   object
 6   sql                  7000 non-null   object
dtypes: object(7)
memory usage: 382.9+ KB


In [7]:
df.isna().sum()
df.isnull().values.any()

False

In [8]:
ip = df['question']
op = df['query']

In [9]:
ip_get = []    
for i in range(730):
  ip_get.append(ip[i])
ip_get

op_get = []    
for i in range(730):
  op_get.append(op[i])
op_get

['SELECT count(*) FROM head WHERE age  >  56',
 'SELECT name ,  born_state ,  age FROM head ORDER BY age',
 'SELECT creation ,  name ,  budget_in_billions FROM department',
 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department',
 'SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15',
 "SELECT name FROM head WHERE born_state != 'California'",
 "SELECT DISTINCT T1.creation FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id JOIN head AS T3 ON T2.head_id  =  T3.head_id WHERE T3.born_state  =  'Alabama'",
 'SELECT born_state FROM head GROUP BY born_state HAVING count(*)  >=  3',
 'SELECT creation FROM department GROUP BY creation ORDER BY count(*) DESC LIMIT 1',
 "SELECT T1.name ,  T1.num_employees FROM department AS T1 JOIN management AS T2 ON T1.department_id  =  T2.department_id WHERE T2.temporary_acting  =  'Yes'",
 'SELECT count(DISTINCT temporary_acting) FROM management',
 'SELECT count(*) FROM department WH

In [10]:
# Removing columns db_id, query_toks, sql, question_tokens, query_toks_no_value as they are of no use.

ctr = 0
data = []
for r in df.itertuples(index = False):
  if 'avg' in r[3] or 'min' in r[3] or 'max' in r[3] or 'ctr' in r[3]:
    ctr += 1
    data.append(r)

df = pd.DataFrame(data)
df_delete = df.drop(['db_id', 'query_toks', 'sql', 'question_toks', 'query_toks_no_value' ], axis = 1)
numbers = df_delete.shape[0]

In [11]:
df=df.dropna()
df_delete.head()

,query,question
0,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
1,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
2,SELECT avg(Working_Horses) FROM farm WHERE Tot...,What is the average number of working horses o...
3,SELECT avg(Working_Horses) FROM farm WHERE Tot...,Give the average number of working horses on f...
4,"SELECT max(Cows) , min(Cows) FROM farm",What are the maximum and minimum number of cow...


In [12]:
ctr = 0
n_table = []
for index, value in df_delete['query'].items():
  n_value = ' ' + value + ' '
  n_table.append(n_value)

all_tables = pd.Series(n_table)
all_df = df_delete.assign(query_column = all_tables)

df_query = all_df.drop(['query'], axis=1)

In [13]:
print(df)
df_query.head()

                     db_id                                              query  \
0    department_management  SELECT max(budget_in_billions) ,  min(budget_i...   
1    department_management  SELECT avg(num_employees) FROM department WHER...   
2                     farm  SELECT avg(Working_Horses) FROM farm WHERE Tot...   
3                     farm  SELECT avg(Working_Horses) FROM farm WHERE Tot...   
4                     farm            SELECT max(Cows) ,  min(Cows) FROM farm   
..                     ...                                                ...   
743        tracking_orders  SELECT order_id FROM shipments WHERE shipment_...   
744           architecture  SELECT max(T1.length_meters) ,  T2.name FROM b...   
745           architecture                SELECT avg(length_feet) FROM bridge   
746        culture_company  SELECT avg(budget_million) ,  max(budget_milli...   
747        culture_company  SELECT avg(budget_million) ,  max(budget_milli...   

                           

,question,query_column
0,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_..."
1,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHE...
2,What is the average number of working horses o...,SELECT avg(Working_Horses) FROM farm WHERE To...
3,Give the average number of working horses on f...,SELECT avg(Working_Horses) FROM farm WHERE To...
4,What are the maximum and minimum number of cow...,"SELECT max(Cows) , min(Cows) FROM farm"


In [14]:
def new_Query(data):
    query = []
    q_ctr = 0
    
    for i, l in data.items():
        for c in l:
            if c not in query:
                query.append(c)
                q_ctr += 1
                
    return q_ctr, query
                
        

def maximum_l(data):
  m = 0

  for i, l in data.items():
    tem = len(l)
    if tem > m:
      m = tem
  
  return m

In [15]:
#Preprossing the data
ques_length = maximum_l(df_query['question'])
query_length = maximum_l(df_query['query_column'])
ques, new_char = new_Query(df_query['question'])
query, search_char = new_Query(df_query['query_column'])

j = dict([(c,i) for i, c in enumerate(new_char)])
k = dict([(c,i) for i, c in enumerate(search_char)])
print(numbers, ques_length, ques)
print(numbers, query_length, query)

748 224 72
748 347 82


In [27]:
def token(i):
  i_tk = Tokenizer()
  i_tk.fit_on_texts(i)
  return i_tk.texts_to_sequences(i), i_tk

def pad(i, length=None):
  if length is None:
    length = max([len(sentence) for sentence in i])
  return pad_sequences(i, maxlen=length, padding='post', truncating='post')

def preprocess(i, j):
  preprocess_i, i_tk = token(i)
  preprocess_j, j_tk = token(j)

  preprocess_i = pad(preprocess_i)
  preprocess_j = pad(preprocess_j)
  preprocess_j = preprocess_j.reshape(*preprocess_j.shape, 1)
  return preprocess_i, preprocess_j, i_tk, j_tk

def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [30]:
preprocess_i, preprocess_j, i_tk, j_tk = preprocess(ip_get, op_get)

i_sequence_length = preprocess_i.shape[1]
j_sequence_length = preprocess_j.shape[1]
i_vocab_size = len(i_tk.word_index)
j_vocab_size = len(j_tk.word_index)

In [31]:
#Splitting data :- Training Data = 70% , Validation data = 15%, Testing Data = 15%
x_train, x_test, y_train, y_test = train_test_split(preprocess_i, preprocess_j, test_size=0.3, random_state=22)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=22)

print(len(x_train), 'train sequences', x_train.shape)
print(len(x_val), 'validation sequences', x_val.shape)
print(len(x_test), 'test sequences', x_test.shape)

511 train sequences (511, 26)
109 validation sequences (109, 26)
110 test sequences (110, 26)


In [ ]:
ec_new_query = np.zeros((numbers, ques_length, ques), dtype = 'float32')
dc_new_query = np.zeros((numbers, query_length, query), dtype = 'float32')
dc_search_query = np.zeros((numbers, query_length, query), dtype = 'float32')
#df_query.head()

In [ ]:
#Endcoder - Decoder Model
init = 0
for new_que, desired_output in df_query.itertuples(index = False):
    for l, c in enumerate(new_que):
        ec_new_query[init, l, j[c]] = 1.0
    ec_new_query[init, l + 1 :, j[' ']] = 1.0
    for l, c in enumerate(desired_output):
        dc_new_query[init, l, k[c]] = 1.0
        if l > 0:
            dc_search_query[init, l, k[c]] = 1.0
        dc_new_query[init, l + 1 :, k[' ']] = 1.0
        dc_search_query[init, l:, k[' ']] = 1.0
    init = init + 1
    if ctr > 5:
        break

i_query = keras.Input(shape=(None, ques))
encode = keras.layers.LSTM(84, return_state = True)
o_query, high, low = encode(i_query)
stat = [high, low]
i_decode = keras.Input(shape=(None, query))
lstm_d = keras.layers.LSTM(84, return_sequences = True, return_state = True)
o_decode, _, _ = lstm_d(i_decode, initial_state=stat)
d_decode = keras.layers.Dense(query, activation = 'softmax')
o_decode = d_decode(o_decode)
k_model = keras.Model([i_query, i_decode], o_decode)

In [ ]:
k_model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics=['accuracy'])
k_model.fit(
    [ec_new_query, dc_new_query], dc_search_query,
    batch_size = 64,
    epochs = 10,
    validation_split=0.2
)

Epoch 1/10
10/10 [==============================] - 11s 678ms/step - loss: 3.3936 - accuracy: 0.7931 - val_loss: 1.7492 - val_accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 6s 561ms/step - loss: 1.7443 - accuracy: 1.0000 - val_loss: 1.8204 - val_accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 6s 599ms/step - loss: 1.8149 - accuracy: 1.0000 - val_loss: 1.8849 - val_accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 6s 564ms/step - loss: 1.8589 - accuracy: 1.0000 - val_loss: 1.9118 - val_accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 6s 564ms/step - loss: 1.8787 - accuracy: 1.0000 - val_loss: 1.9295 - val_accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 6s 562ms/step - loss: 1.8930 - accuracy: 1.0000 - val_loss: 1.9395 - val_accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 6s 563ms/step - loss: 1.9024 - accuracy: 1.0000 - val_loss: 1.9491 - val_accuracy: 1.0000
Epoch

In [44]:
def encdec_model(ip_shape, op_sequence, x_vocab_size, y_vocab_size):
    learning_rate = 0.001
    mdl = Sequential()
    ipt = Input(shape=ip_shape[1:])
    gru = GRU(op_sequence)(ipt)
    e_out = Dense(128, activation='relu')(gru)
    d_input = RepeatVector(op_sequence)(e_out)
    d_gru = GRU(128, return_sequences=True)(d_input)
    layer = TimeDistributed(Dense(y_vocab_size, activation='softmax'))
    final = layer(d_gru)
    mdl = Model(inputs=ipt, outputs=final)
    mdl.compile(loss=sparse_categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])   
    return mdl

tmp_x = pad(x_train, y_train.shape[1])
tmp_x = tmp_x.reshape((-1, y_train.shape[-2], 1))
x_val_processed = pad(x_val, y_val.shape[1])
x_val_processed = x_val_processed.reshape((-1, y_val.shape[-2], 1)) 
ed_rnn_model = encdec_model(
    tmp_x.shape,
    y_train.shape[1],
    len(x_tk.word_index)+1,
    len(y_tk.word_index)+1)
history =ed_rnn_model.fit(tmp_x, y_train, batch_size=20, epochs=100, validation_data=(x_val_processed, y_val))

Epoch 1/100
26/26 [==============================] - 8s 154ms/step - loss: 4.0146 - accuracy: 0.7500 - val_loss: 1.5259 - val_accuracy: 0.7875
Epoch 2/100
26/26 [==============================] - 3s 123ms/step - loss: 1.6105 - accuracy: 0.7500 - val_loss: 1.3109 - val_accuracy: 0.7875
Epoch 3/100
26/26 [==============================] - 3s 119ms/step - loss: 1.4928 - accuracy: 0.7500 - val_loss: 1.2911 - val_accuracy: 0.7875
Epoch 4/100
26/26 [==============================] - 3s 124ms/step - loss: 1.4605 - accuracy: 0.7500 - val_loss: 1.2363 - val_accuracy: 0.7875
Epoch 5/100
26/26 [==============================] - 3s 121ms/step - loss: 1.4160 - accuracy: 0.7635 - val_loss: 1.1948 - val_accuracy: 0.8073
Epoch 6/100
26/26 [==============================] - 3s 121ms/step - loss: 1.3828 - accuracy: 0.7660 - val_loss: 1.1781 - val_accuracy: 0.8067
Epoch 7/100
26/26 [==============================] - 3s 123ms/step - loss: 1.3552 - accuracy: 0.7698 - val_loss: 1.1716 - val_accuracy: 0.8101

NameError: ignored

In [43]:
print('Accuracy =', np.average(history_bd.history['accuracy']))
print('Loss = ', np.average(history_bd.history['loss']))
print('Validation accuracy =', np.average(history_bd.history['val_accuracy']))
print('Validation loss =', np.average(history_bd.history['val_loss']))

NameError: ignored